In [6]:
import io
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os

# Аутентификация
folder_id = '1AUBzs9AnfmZq8u0MiXcsb96C7X5bjuzG'
credentials = Credentials.from_service_account_file('cred.json', scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=credentials)

results = drive_service.files().list(
    q=f"'{folder_id}' in parents",
    fields="nextPageToken, files(id, name)"
).execute()

items = results.get('files', [])

# Проверка и создание локальной папки 'test_videos' если не существует
if not os.path.exists('test_videos'):
    os.makedirs('test_videos')

if not items:
    print('Нет файлов в папке.')
else:
    for number in range(51, 100):
        file_name = f"{number}_144.mp4"
        
        # Поиск файла по имени и родительской папке
        results = drive_service.files().list(
        q=f"name='{file_name}' and '{folder_id}' in parents",
        fields="files(id, name)"
        ).execute()
        
        items = results.get('files', [])
        
        if not items:
            print(f"Файл {file_name} не найден.")
            continue
        
        item = items[0]
        file_id = item['id']
        
        # Загрузка файла
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        
        # Сохранение файла в локальную папку
        with open(f'test_videos/{file_name}', 'wb') as f:
            f.write(fh.getvalue())
        
        print(f"Файл {file_name} (ID: {file_id}) успешно загружен в папку 'test_videos'.")

Файл 51_144.mp4 (ID: 1ThMQHmsYnT-1xUm9LpYG5PEamPTpbqfs) успешно загружен в папку 'test_videos'.
Файл 52_144.mp4 (ID: 1A9nNg-gZFKwHMlhdVLJF0r2YxcSic2ZQ) успешно загружен в папку 'test_videos'.
Файл 53_144.mp4 (ID: 1RJXgNCSBbCdh-muR9Oj16FQaEOrffcPp) успешно загружен в папку 'test_videos'.
Файл 54_144.mp4 (ID: 1xiZMRs5l8UkRzHfNOHSNOCQOZIPkpR31) успешно загружен в папку 'test_videos'.
Файл 55_144.mp4 (ID: 15Iu08r2DR8psP7-A20yYOYaySpb7QgR_) успешно загружен в папку 'test_videos'.
Файл 56_144.mp4 (ID: 1d7nGgiGnkfr2R2xBnjn4CoER9mKq8tsW) успешно загружен в папку 'test_videos'.
Файл 57_144.mp4 (ID: 15X-VxxF0Iv5VK1-LhNk8zTmZl3fIL1Mg) успешно загружен в папку 'test_videos'.
Файл 58_144.mp4 (ID: 1O364cjnjd03MXwiaBLSU2NP6Ff-Kcf46) успешно загружен в папку 'test_videos'.
Файл 59_144.mp4 (ID: 14YRGJFXQU9sC5ZcfBETifrAFIgucrMOd) успешно загружен в папку 'test_videos'.
Файл 60_144.mp4 (ID: 1mFR4RqRrjT_y3_bNIT9GpkyaI8B5_FK3) успешно загружен в папку 'test_videos'.
Файл 61_144.mp4 (ID: 1a2XJ0U9O9gS5NjBY8X

In [7]:
import re
import shutil
import subprocess
import time

import cv2
from moviepy.editor import *

In [8]:
# Функция для извлечения кадров из видео
def extract_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    fps = cap.get(cv2.CAP_PROP_FPS)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        
        if frame_count % fps != 0:
            frame_count += 1
            continue


        frame_file = f"{output_folder}/frame_{frame_count}.jpg"

        height, width, layers = frame.shape
        target_height = 480
        scaling_factor = target_height / float(height)
        target_width = int(float(width) * scaling_factor)
        
        # Изменение размера с использованием интерполяции
        resized_image = cv2.resize(frame, (target_width, target_height), interpolation=cv2.INTER_CUBIC)
        
        cv2.imwrite(frame_file, resized_image)
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [9]:
# Функция для сортировки файлов по натуральному порядку
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

In [10]:
# Функция для превращения кадров в видео
def frames_to_video_with_audio(input_folder, output_video, fps):
    images = [img for img in os.listdir(input_folder) if img.endswith(".jpg")]

    frame = cv2.imread(os.path.join(input_folder, images[0]))

    height, width, layers = frame.shape
    video = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(input_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [11]:
# Функция удаления всех файлов и папок в папке
def delete_all_files_in_folder(folders: list):
    for folder in folders:
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")

        for root, dirs, files in os.walk(folder, topdown=False):
            for name in dirs:
                os.rmdir(os.path.join(root, name))

In [12]:
# Пути и параметры

# Путь к видео
video_path = "./test_videos/"

# Путь к папке для GFPGAN фреймов, для обработки 
output_folder_frames = "./input_videos"

# Путь к корневой папке с готовыми фреймами
save_frames_folder_frames = "./results_videos"

# Путь к папке с готовыми фреймами, после обработки GFPGAN
input_folder_frames = save_frames_folder_frames + "/restored_imgs"

# Путь для сохранения итогового видео 
output_video = "./done/output_video_with_audio1.mp4"

# Путь к файлу со скриптом VENV
# venv_python_path = "C:\\Users\meyap\PycharmProjects\VideoPlus\\venv\Scripts\python.exe"

fps = 1
scale_level = "3"

# Инициализация таймеров
time_extract_frames = 0
time_upscale = 0
time_frames_to_video = 0

In [13]:
# Удаляем все файлы в папке с кадрами
delete_all_files_in_folder([output_folder_frames, input_folder_frames])

In [ ]:
# Извлекаем кадры
start_time = time.time()
for index in range(52, 100):
    extract_frames(video_path + str(index) + "_144.mp4", output_folder_frames)
    time_extract_frames = time.time() - start_time
    # Запускаем внешний скрипт для апскейла
    start_time = time.time()
    subprocess.run(["python3", "./GFPGAN/inference_gfpgan.py", "-i",
                    output_folder_frames, "-o", save_frames_folder_frames, "-v", "1.3", "-s", scale_level])
    time_upscale = time.time() - start_time
    start_time = time.time()
    frames_to_video_with_audio(input_folder_frames, output_video, int(fps))
    time_frames_to_video = time.time() - start_time
    print(f"Time for extracting frames: {time_extract_frames:.2f} seconds")
    print(f"Time for upscaling: {time_upscale:.2f} seconds")
    print(f"Time for converting frames to video: {time_frames_to_video:.2f} seconds")
    print(f"Total time: {time_extract_frames + time_upscale + time_frames_to_video:.2f} seconds")
    from google.oauth2.service_account import Credentials
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaFileUpload
    
    # Аутентификация
    # Аутентификация, как раньше
    # Аутентификация
    credentials = Credentials.from_service_account_file('cred.json', scopes=['https://www.googleapis.com/auth/drive'])
    drive_service = build('drive', 'v3', credentials=credentials)
    folder_id = '1pQmoecRBA8z6Nur8WFwO-fK560tA8iOl'

    # Инфо о файле
    file_metadata = {
        'name': str(index) + "_144.mp4",
        'mimeType': 'video/mp4',
        'parents': [folder_id]  # Добавление ID папки

    }
    
    # Загрузка файла
    media = MediaFileUpload('video.mp4', mimetype='video/mp4')
    request = drive_service.files().create(media_body=media, body=file_metadata)
    response = request.execute()
    
    # Делаем файл общедоступным
    file_id = response['id']
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }
    drive_service.permissions().create(fileId=file_id, body=permission).execute()
    
    print(f"Файл загружен и сделан общедоступным, ID: {file_id}")
    # Удаляем все файлы в папке с кадрами
    delete_all_files_in_folder([output_folder_frames, input_folder_frames])
